In [93]:
import numpy as np
import pandas as pd
import os
import sys
from glob import glob
from tqdm.notebook import tqdm



# 커피지수 part_1

In [109]:
data_dir = '/content/drive/MyDrive/데이터/소상공인시장진흥공단_상가(상권)정보_20211231'
sys.path.insert(0, data_dir)

In [95]:
data_dir=data_dir+'/소상공인시장진흥공단_상가(상권)정보_서울_202112.csv'
df = pd.read_csv(data_dir)
df.head(2)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,23324279,제중건강원,NaN,D,소매,D10,건강/미용식품,D10A07,건강원,G47216,...,1156010600100410002034626,NaN,서울특별시 영등포구 영중로14길 11-17,150035,7250.0,NaN,1,NaN,126.907168,37.520613
1,24525909,민속악기사,NaN,D,소매,D04,취미/오락관련소매,D04A09,악기판매,G47593,...,1120012200101420011000227,NaN,서울특별시 성동구 용답5길 2,133849,4803.0,NaN,NaN,NaN,127.049018,37.566857


In [96]:
# 업종중분류코드 : Q12 --> 카페/커피샵
df = df[df.상권업종중분류코드 == 'Q12']

In [97]:
df = df[['상호명', '지점명', '시도명', '시군구명']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18628 entries, 23 to 315277
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상호명     18628 non-null  object
 1   지점명     6918 non-null   object
 2   시도명     18628 non-null  object
 3   시군구명    18628 non-null  object
dtypes: object(4)
memory usage: 727.7+ KB


In [98]:
df.head()

,상호명,지점명,시도명,시군구명
23,커피빈,코리아대학로대명거리점,서울특별시,종로구
99,요거프레소,쌍문점,서울특별시,도봉구
191,우성커피숍,NaN,서울특별시,양천구
198,버블베어,NaN,서울특별시,강서구
233,알뤼르,NaN,서울특별시,강남구


In [99]:
#df['상호명'].unique()

# 스타벅스 커피숍
# case=False : 대소문자 무시하고 다 찾아라
sb = df[ df.상호명.str.contains('스타벅스|STARBUCKS', case=False) ]
sb.head(3)

,상호명,지점명,시도명,시군구명
1544,스타벅스,동숭로아트점,서울특별시,종로구
2474,스타벅스남부터미널2점,남부터미널2점,서울특별시,서초구
2570,스타벅스,현대목동점,서울특별시,양천구


In [100]:
# 지점명중 스타벅스지점명이 없는 데이터 검색
df[ df.지점명.str.contains('스타벅스|STARBUCKS', case=False, na=False) ]

,상호명,지점명,시도명,시군구명


In [101]:
# 커피빈 이디야 빽다방의 한/영문 같이검색 cb, ed, bd
cb = df[ df.상호명.str.contains('커피빈|COFFEEBEAN', case=False) ]
ed = df[ df.상호명.str.contains('이디야|EDIYA', case=False) ]
bd = df[ df.상호명.str.contains('빽다방|PAIKSDCOFFEE', case=False) ]
bd.head()

,상호명,지점명,시도명,시군구명
7179,빽다방공덕새창로점,공덕새창로점,서울특별시,마포구
7789,빽다방서초우성점,서초우성점,서울특별시,서초구
18660,빽다방중계은행사거리점,중계은행사거리점,서울특별시,노원구
24413,빽다방성신여대점,성신여대점,서울특별시,성북구
24973,빽다방신림역1호점,신림역1호점,서울특별시,관악구


In [102]:
len(sb), len(cb), len(ed), len(bd)

(467, 148, 416, 141)

# 전국 데이터에서 4개 매장 정보 추출

In [103]:
# 빈 데이터 프레임을 만들기
starbucks = pd.DataFrame(columns=['상호명','시도명','시군구명'])
coffeebean = pd.DataFrame(columns=['상호명','시도명','시군구명'])
ediya = pd.DataFrame(columns=['상호명','시도명','시군구명'])
paik = pd.DataFrame(columns=['상호명','시도명','시군구명'])

In [110]:
 #디렉토리내 데이터파일을 for문으로 불러와 전처리
for file in tqdm(glob(f'{data_dir}/*.csv')):
        print(file)
        df = pd.read_csv(file, low_memory=False)
        df = df[ df.상권업종중분류코드 == 'Q12' ]
        df = df[ ['상호명','시도명','시군구명'] ]
        sb = df[ df.상호명.str.contains('스타벅스|STARBUCKS', case=False) ]
        cb = df[ df.상호명.str.contains('커피빈|COFFEEBEAN', case=False) ]
        ed = df[ df.상호명.str.contains('이디야|EDIYA', case=False) ]
        bd = df[ df.상호명.str.contains('빽다방|PAIKSCOFFEE', case=False) ]

        #print(sb)

        starbucks = pd.concat( [starbucks, sb] )
        coffeebean = pd.concat([coffeebean, cb])
        ediya = pd.concat([ediya, ed])
        paik = pd.concat([paik, bd])

  0%|          | 0/17 [00:00<?, ?it/s]

/content/drive/MyDrive/데이터/소상공인시장진흥공단_상가(상권)정보_20211231/소상공인시장진흥공단_상가(상권)정보_강원_202112.csv
/content/drive/MyDrive/데이터/소상공인시장진흥공단_상가(상권)정보_20211231/소상공인시장진흥공단_상가(상권)정보_경기_202112.csv
/content/drive/MyDrive/데이터/소상공인시장진흥공단_상가(상권)정보_20211231/소상공인시장진흥공단_상가(상권)정보_경남_202112.csv
/content/drive/MyDrive/데이터/소상공인시장진흥공단_상가(상권)정보_20211231/소상공인시장진흥공단_상가(상권)정보_광주_202112.csv
/content/drive/MyDrive/데이터/소상공인시장진흥공단_상가(상권)정보_20211231/소상공인시장진흥공단_상가(상권)정보_경북_202112.csv
/content/drive/MyDrive/데이터/소상공인시장진흥공단_상가(상권)정보_20211231/소상공인시장진흥공단_상가(상권)정보_대구_202112.csv
/content/drive/MyDrive/데이터/소상공인시장진흥공단_상가(상권)정보_20211231

In [111]:
starbucks.shape, coffeebean.shape, ediya.shape, paik.shape

((1504, 3), (257, 3), (2076, 3), (727, 3))

In [112]:
starbucks.head()

,상호명,시도명,시군구명
2920,스타벅스강릉안목항점,강원도,강릉시
6507,스타벅스춘천후평DT점,강원도,춘천시
8720,스타벅스,강원도,춘천시
9813,스타벅스설악워터피아점,강원도,속초시
10522,스타벅스,강원도,원주시


In [113]:
starbucks.to_csv('./스타벅스.csv', index=False)
coffeebean.to_csv('./커피빈.csv', index=False)
ediya.to_csv('./이디야.csv', index=False)
paik.to_csv('./빽다방.csv', index=False)